# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///MessagesCleanDatabase.db')
df = pd.read_sql("SELECT * FROM LabeledMessages", engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Normalize text 
    text = text.lower() 
    text = text.strip()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Tokenize text 
    tokens = word_tokenize(text) 
    
    # Remove stop words 
    tokens = [t for t in tokens if t not in stopwords.words("english")] 
    
    # Lemmatization - Reduce words to their root form 
    tokens = [WordNetLemmatizer().lemmatize(t) for t in tokens]
    
    # Stemming - Reduce words to their stems
#     tokens = [PorterStemmer().stem(t) for t in tokens] 
    
    clean_tokens = [] 
    for tok in tokens: 
#         clean_tok = lemmatizer.lemmatize(tok).lower().strip() 
        clean_tokens.append(tok) 
    return clean_tokens
print(tokenize(X[0]))
print(tokenize(X[1]))
print(tokenize(X[26203]))

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
['hurricane']
['bangkok', '24', 'january', '2012', 'nnt', 'prime', 'minister', 'yingluck', 'shinawatra', 'attended', 'meeting', 'permanent', 'secretary', 'various', 'ministry', 'urging', 'quickly', 'distribute', 'flood', 'compensation', 'wisely', 'utilize', 'budget']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier 
pipeline.fit(X_train, y_train) 

# predict on test data 
y_pred = pipeline.predict(X_test) 

In [6]:
print(y_pred)

[[1 0 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]]


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
# print(classification_report(y_test,y_pred))

labels = np.unique(y_pred) 
# confusion_mat = confusion_matrix(y_test, y_pred, labels=labels) 
accuracy = (y_pred == y_test).mean() 
# print("Labels:", labels) 
# print("Confusion Matrix:\n", confusion_mat) 
print("Accuracy:", accuracy)

Accuracy: related                   0.799664
request                   0.883735
offer                     0.994812
aid_related               0.741684
medical_help              0.927525
medical_products          0.953769
search_and_rescue         0.974214
security                  0.985047
military                  0.971315
child_alone               1.000000
water                     0.953158
food                      0.931950
shelter                   0.933171
clothing                  0.984132
money                     0.976655
missing_people            0.988862
refugees                  0.968264
death                     0.960635
other_aid                 0.869240
infrastructure_related    0.927830
transport                 0.951175
buildings                 0.951175
electricity               0.977571
tools                     0.992981
hospitals                 0.988404
shops                     0.994507
aid_centers               0.987489
other_infrastructure      0.950717
weather_re

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {
    'vect__min_df': [1, 5],
    'tfidf__use_idf':[True, False],
    'clf__estimator__n_estimators':[10, 25], 
    'clf__estimator__min_samples_split':[2, 5, 10]
}

cv = SVC()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.